In [1]:
import targets as targets
import classification as classif
import stage as stage
import rules as rules
import comms as comms
import processor as processor
import datetime

In [2]:
ts = targets.TargetSpace()

In [3]:
bc = classif.BackgroundClassifier()
rnc = classif.RadiusNeighborsClassifier(classif.classification_weights, ts, outliers=bc)

In [4]:
send_triggers = rules.SendTriggers()

In [5]:
classification_processor = processor.ClassificationProcessor(rnc, ts, send_triggers, True)

In [6]:
stage_instance = stage.Stage(classification_processor, ts, send_triggers)

In [7]:
comms_instance = comms.InstrumentComms(stage_instance, True)

In [ ]:
#For testing, uncomment 'while True' in stage.Stage.processEligibleStagedData()
#stage_instance.processEligibleStagedData()

In [ ]:
#For testing
#stage_instance.recent_targets

In [8]:
stage_instance.processEligibleStagedData()

type:  <class 'list'>
np.squeeze(neigh_ind.tolist()).tolist():  []
Classified target <targets.Target object at 0x000002424A1F81D0>, classification: 1
type:  <class 'list'>
np.squeeze(neigh_ind.tolist()).tolist():  []
Classified target <targets.Target object at 0x000002424A1F8128>, classification: 1
type:  <class 'list'>
np.squeeze(neigh_ind.tolist()).tolist():  []
Classified target <targets.Target object at 0x000002424A1F8198>, classification: 1
type:  <class 'list'>
np.squeeze(neigh_ind.tolist()).tolist():  []
Classified target <targets.Target object at 0x000002424A1F80B8>, classification: 1
type:  <class 'list'>
np.squeeze(neigh_ind.tolist()).tolist():  []
Classified target <targets.Target object at 0x000002424A1F8208>, classification: 1
type:  <class 'list'>
np.squeeze(neigh_ind.tolist()).tolist():  []
Classified target <targets.Target object at 0x000002424A1F8208>, classification: 1
type:  <class 'list'>
np.squeeze(neigh_ind.tolist()).tolist():  []
Classified target <targets.Target

KeyboardInterrupt: 

In [17]:
stage_instance.target_space.tables

{'adcp': [[datetime.datetime(2016, 5, 16, 11, 43, 5), 1.2, 4.5]],
 'camera': [],
 'classifier_classifications': ['1.1',
  '1.2',
  '1.1',
  '1.2',
  '1.2',
  '1.2',
  '1.2',
  '1.1'],
 'classifier_features': [[0.3333333333333333,
   0.4,
   0.75,
   0.02666666666666667,
   0.25,
   1.75],
  [2.3333333333333335,
   1.2,
   1.25,
   0.02666666666666667,
   0.08333333333333333,
   1.25],
  [0.5, 0.4, 0.75, 0.02666666666666667, 0.25, 1.75],
  [2.3333333333333335,
   1.2,
   1.25,
   0.02666666666666667,
   0.08333333333333333,
   1.25],
  [2.3333333333333335,
   1.2,
   1.25,
   0.02666666666666667,
   0.09166666666666667,
   1.275],
  [2.3333333333333335,
   1.2,
   1.25,
   0.02666666666666667,
   0.08750000000000001,
   1.25],
  [0.5, 0.44000000000000006, 0.775, 0.02666666666666667, 0.25, 1.75],
  [0.5, 0.4, 0.75, 0.02666666666666667, 0.20833333333333334, 1.75]],
 'nims': [],
 'pamguard': []}

In [13]:
import sqlite3
conn = sqlite3.connect('current_model_targets.db')
c = conn.cursor()
c.execute('''CREATE TABLE current_model_targets 
(id,size,zpeed,deltav,target_strength,current,time_of_day,passive_acoustics,
source,date,classification)''')

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Emma\Anaconda3\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\Emma\Anaconda3\lib\threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Emma\Documents\NNMREC\AMP Interface\data-science-incubator\ARTEMIS\ARTEMIS\comms_NIMS_sim.py", line 48, in read_tracks
    s.connect((host, port))
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

ERROR: An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 27))



OperationalError: table current_model_targets already exists

In [14]:
from config import site_name

In [15]:
def update_classifier_database(c, stage_instance):
    """
    Enter current model values into database (with cursor c)
    """
    ident = [1] # deal with this
    site = [site_name + '_auto']

    entries = []
    for i, feat_list in enumerate(stage_instance.target_space.tables['classifier_features']):
        feats = stage_instance.target_space.tables['classifier_features'][i]
        class_placeholder = [0]
        date_placeholder = [0]
        passive_placeholder = [0]
        entry = tuple(ident + feats + passive_placeholder + site + date_placeholder + class_placeholder)
        entries.append(entry)
            
    c.executemany("INSERT INTO current_model_targets VALUES (?,?,?,?,?,?,?,?,?,?,?)", entries)

In [16]:
update_classifier_database(c,stage_instance)

[(1, 0.3333333333333333, 0.4, 0.75, 0.02666666666666667, 0.25, 1.75, 0, 'MSL_auto', 0, 0), (1, 2.3333333333333335, 1.2, 1.25, 0.02666666666666667, 0.08333333333333333, 1.25, 0, 'MSL_auto', 0, 0), (1, 0.5, 0.4, 0.75, 0.02666666666666667, 0.25, 1.75, 0, 'MSL_auto', 0, 0), (1, 2.3333333333333335, 1.2, 1.25, 0.02666666666666667, 0.08333333333333333, 1.25, 0, 'MSL_auto', 0, 0), (1, 2.3333333333333335, 1.2, 1.25, 0.02666666666666667, 0.09166666666666667, 1.275, 0, 'MSL_auto', 0, 0), (1, 2.3333333333333335, 1.2, 1.25, 0.02666666666666667, 0.08750000000000001, 1.25, 0, 'MSL_auto', 0, 0), (1, 0.5, 0.44000000000000006, 0.775, 0.02666666666666667, 0.25, 1.75, 0, 'MSL_auto', 0, 0), (1, 0.5, 0.4, 0.75, 0.02666666666666667, 0.20833333333333334, 1.75, 0, 'MSL_auto', 0, 0)]


In [65]:
a = [4,2,3]

In [66]:
tuple(a)

(4, 2, 3)

In [68]:
for i,it in enumerate(a):
    print(i)

0
1
2
